In [ ]:
class Entity

class Revision:
    
    def __init__(self, title, entity_json):
        self.title = title
        self.entity_json = entity_json

In [1]:
from lxml import etree
from io import BytesIO, StringIO

import json
import os

base = '{http://www.mediawiki.org/xml/export-0.10/}'

In [ ]:
import collections

entities = collections.Counter()
context = etree.iterparse('test.xml')

for action, elem in context:
    tag = elem.tag
    if tag == f"{base}title":
        title = elem.text
    elif tag == f"{base}revision":
        entities.update([title])
    else:
        elem.clear()


In [ ]:
list(entities.most_common(5000))[:-10:-1]

In [ ]:
objects = list()
context = etree.iterparse('wikidata_a.xml', huge_tree=True)

# turn it into an iterator
context = iter(context)

# get the root element
event, root = next(context)

entity_to_serialize = 'Q108695611'
wd_item = False
valid_entity = False
timestamp = ''
error = False

for action, elem in context:
    tag = elem.tag
    if tag == f"{base}title":
        title = elem.text
#        if title in entities_to_serialize:
#            print("VALID ENTITY")
#            print(title)
#            valid_entity = True
#        else:
#            valid_entity = False
    elif tag == f"{base}format":# and valid_entity:
        wd_item = elem.text == 'application/json'
    elif tag == f"{base}timestamp":# and valid_entity:
        timestamp = elem.text
    elif tag == f"{base}text" and wd_item:# and valid_entity:
        try:
            entity_json = json.loads(elem.text.encode('utf-8'))
        except Exception as e:
            print("ERROOOOOOOOOR")
            print(elem.text)
            print(e)
            error = True
    else:
        elem.clear()
        root.clear()


# Performance test

In [2]:
from jsondiff import diff

def do_stuff():
    objects = list()
    context = etree.iterparse('wikidata_a.xml', huge_tree=True)

    # turn it into an iterator and get the root element
    context = iter(context)
    event, root = next(context)

    wd_item = False
    valid_entity = False
    timestamp = ''
    i = 0
    
    prev_json = None

    for action, elem in context:
        tag = elem.tag
        if tag == f"{base}title":
            title = elem.text
            print(title)

            i += 1
            if i == 15:
                break
        elif tag == f"{base}format":# and valid_entity:
            wd_item = elem.text == 'application/json'
        elif tag == f"{base}timestamp":# and valid_entity:
            timestamp = elem.text
        elif tag == f"{base}text" and wd_item:# and valid_entity:            
            try:
                entity_json = json.loads(elem.text.encode('utf-8'))
                
                if prev_json is None:
                    # TODO: upload entire current json
                    pass
                else:
                    d = diff(prev_json, entity_json)
                prev_json = entity_json
            except Exception as e:
                print("ERROOOOOOOOOR")
                print(elem.text)
                print(e)
        else:
            elem.clear()
            root.clear()
    return objects



In [4]:
%load_ext snakeviz

In [6]:
%snakeviz do_stuff()

Wikidata:Main Page/Content
MediaWiki:Mainpage
User:Reedy
User:Denny Vrandečić (WMDE)
User:Katie Filbert (WMDE)
User:Denny
User:Jeblad/archive 1
Q15
MediaWiki:Sidebar
Q17


KeyboardInterrupt: 